In [1]:
import numpy as np
import pandas as pd

In [4]:
url = 'https://github.com/mattharrison/datasets/raw/master/data/siena2018-pres.csv'

In [5]:
df = pd.read_csv(url, index_col=0)

In [6]:
def tweak_siena_pres(df):
    def int64_to_uint8(df_):
        cols = df_.select_dtypes('int64')
        return (df_
                .astype({col:'uint8' for col in cols}))


    return (df
     .rename(columns={'Seq.':'Seq'})    # 1
     .rename(columns={k:v.replace(' ', '_') for k,v in
        {'Bg': 'Background',
         'PL': 'Party leadership', 'CAb': 'Communication ability',
         'RC': 'Relations with Congress', 'CAp': 'Court appointments',
         'HE': 'Handling of economy', 'L': 'Luck',
         'AC': 'Ability to compromise', 'WR': 'Willing to take risks',
         'EAp': 'Executive appointments', 'OA': 'Overall ability',
         'Im': 'Imagination', 'DA': 'Domestic accomplishments',
         'Int': 'Integrity', 'EAb': 'Executive ability',
         'FPA': 'Foreign policy accomplishments',
         'LA': 'Leadership ability',
         'IQ': 'Intelligence', 'AM': 'Avoid crucial mistakes',
         'EV': "Experts' view", 'O': 'Overall'}.items()})
     .astype({'Party':'category'})  # 2
     .pipe(int64_to_uint8)  # 3
     .assign(Average_rank=lambda df_:(df_.select_dtypes('uint8') # 4
                 .sum(axis=1).rank(method='dense').astype('uint8')),
             Quartile=lambda df_:pd.qcut(df_.Average_rank, 4,
                 labels='1st 2nd 3rd 4th'.split())
            )
    )

In [7]:
pres = tweak_siena_pres(df)

In [9]:
scores = (pres
         .loc[:,'Background':'Average_rank'])

In [10]:
scores

,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,Leadership_ability,Communication_ability,...,Relations_with_Congress,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank
1,7,7,1,10,1,6,2,2,1,11,...,1,1,1,1,2,2,1,2,1,1
2,3,13,4,4,24,14,31,21,21,13,...,17,4,13,15,19,13,16,10,14,13
3,2,2,14,1,8,5,14,6,6,4,...,5,7,20,4,6,9,7,5,5,5
4,4,6,7,3,16,15,6,13,17,10,...,10,6,14,7,11,19,11,8,7,7
5,9,14,11,18,6,16,7,10,12,15,...,8,11,9,9,10,5,6,9,8,8
6,1,9,6,5,29,19,24,22,23,12,...,29,15,17,18,21,15,14,18,18,18
7,37,15,29,28,4,4,38,11,9,18,...,16,30,25,25,17,23,20,19,19,19
8,23,22,27,25,34,28,20,28,27,25,...,23,25,31,26,29,27,24,28,25,25
9,22,38,28,37,44,32,41,38,29,31,...,37,42,41,40,42,44,37,39,39,38
10,34,33,35,34,22,26,37,36,37,34,...,40,38,34,36,36,26,32,36,37,37


In [11]:
s1 = scores.iloc[:3, :4]

In [12]:
s1

,Background,Imagination,Integrity,Intelligence
1,7,7,1,10
2,3,13,4,4
3,2,2,14,1


In [13]:
s2 = scores.iloc[1:6, :5]

In [14]:
s2

,Background,Imagination,Integrity,Intelligence,Luck
2,3,13,4,4,24
3,2,2,14,1,8
4,4,6,7,3,16
5,9,14,11,18,6
6,1,9,6,5,29


In [15]:
s1 + s2

,Background,Imagination,Integrity,Intelligence,Luck
1,NaN,NaN,NaN,NaN,NaN
2,6.0,26.0,8.0,8.0,NaN
3,4.0,4.0,28.0,2.0,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN


In [16]:
scores.iloc[:3, :4] + pd.concat([scores.iloc[1:6, :5]]*2)

,Background,Imagination,Integrity,Intelligence,Luck
1,NaN,NaN,NaN,NaN,NaN
2,6.0,26.0,8.0,8.0,NaN
2,6.0,26.0,8.0,8.0,NaN
3,4.0,4.0,28.0,2.0,NaN
3,4.0,4.0,28.0,2.0,NaN
4,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN


In [17]:
pd.concat([scores.iloc[1:6, :5]]*2).index.duplicated().any()

True